In [1]:
import sys
import importlib.metadata

REQUIRED_PACKAGES = {
    "opencv-python": "4.9.0.80",
    "numpy": "1.23.5",
    "tensorflow": "2.12.0"
}
REQUIRED_PACKAGES_NO_VERSION_DEPENDENCY = [
    "CodingRider",
    "keyboard"
]

print("환경 검증 시작...")

try:
    for package, required_version in REQUIRED_PACKAGES.items():
        installed_version = importlib.metadata.version(package)
        if not installed_version.startswith(required_version):
            raise ImportError(
                f"'{package}'의 버전이 잘못되었습니다. "
                f"필요 버전: {required_version}, 설치된 버전: {installed_version}"
            )
        print(f" - {package} {installed_version} (요구사항: {required_version}) - 확인 완료")
    for package in REQUIRED_PACKAGES_NO_VERSION_DEPENDENCY:
            installed_version = importlib.metadata.version(package)
            print(f" - {package} {installed_version} (설치 확인) - 확인 완료")
except importlib.metadata.PackageNotFoundError as e:
    raise type(e)(
        f"{e}\n\n"
        "필요한 패키지가 설치되지 않았습니다.\n"
        "VS Code 우측 상단의 커널 선택(Select Kernel) 버튼을 눌러 'tensorflow (2.12.0)'을 선택해주세요."
    ).with_traceback(e.__traceback__) from None
except ImportError as e:
    raise type(e)(
        f"{e}\n\n"
        "잘못된 커널이 선택되었습니다\n"
        "VS Code 우측 상단의 커널 선택(Select Kernel) 버튼을 눌러 'tensorflow (2.12.0)'을 선택해주세요."
    ).with_traceback(e.__traceback__) from None

print("모든 환경 검증 완료. 코드를 계속 실행하세요.")

환경 검증 시작...
 - opencv-python 4.9.0.80 (요구사항: 4.9.0.80) - 확인 완료
 - numpy 1.23.5 (요구사항: 1.23.5) - 확인 완료
 - tensorflow 2.12.0 (요구사항: 2.12.0) - 확인 완료
 - CodingRider 2.9 (설치 확인) - 확인 완료
 - keyboard 0.13.5 (설치 확인) - 확인 완료
모든 환경 검증 완료. 코드를 계속 실행하세요.


In [ ]:
# 포트 확인 후 알맞은 포트 이름을 입력해야 정상적으로 작동
# 예제는 실험 환경이었던 COM4로 진행
# 장치 관리자(Device manager)의 포트(Port) 항목에서 연결된 기기 확인 가능
OPEN_PORT = 'COM4'

In [ ]:
import copy 
from keras.models import load_model  
import cv2 
import numpy as np
from time import sleep
import keyboard
from CodingRider.drone import *
from CodingRider.protocol import *

drone = Drone()
drone.open('COM4')

# is_takeoff 변수로 이륙했는지 확인합니다.
is_takeoff = False
np.set_printoptions(suppress=True)
model = load_model('qnpfr/keras_model.h5', compile=False)
class_names = open('qnpfr/labels.txt', 'r', encoding='UTF-8').readlines()
capture = cv2.VideoCapture(0)

while capture.isOpened():    
    ret, frame = capture.read()
    if not ret: 
        continue
    frame = cv2.flip(frame, 1)   
    frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)      
    origin_frame = copy.deepcopy(frame) 
    frame = np.asarray(frame, dtype=np.float32).reshape(1, 224, 224, 3)
    frame = (frame / 127.5) - 1    
    prediction = model.predict(frame,verbose=0)    
    index = np.argmax(prediction)   
    class_name = class_names[index]
    confidence_score = prediction[0][index] 
    class_name = class_name[2:].strip() 
    confidence_score = round(confidence_score * 100, 2)
    text = f'{class_name} : {confidence_score}%'
    
    # <엔터> 키를 누르면 영상을 분석합니다. 
    if keyboard.is_pressed('enter'):
        # 0번 클래스(takeoff)로 분류했고 이륙하지 않았다면 이륙합니다.
        if (index == 0 and not is_takeoff):
            cv2.putText(origin_frame, text, (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 0))
            cv2.imshow('Machine Learning', origin_frame)
            print('이륙')       
            sleep(2)
            drone.sendTakeOff()
            sleep(5)
            print('이륙 완료')           
            is_takeoff = True
        # 1번 클래스(landing)로 분류했고 이륙했다면 착륙합니다.
        if (index == 1 and is_takeoff):
            cv2.putText(origin_frame, text, (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 0, 0))
            cv2.imshow('Machine Learning', origin_frame)
            print('착륙')
            drone.sendControlWhile(0, 0, 0, 0, 500) # 제자리에서 0.5초 멈춥니다.
            drone.sendLanding()
            sleep(3)
            print('착륙 완료') 
            is_takeoff = False
    # 안전을 위해서 키보드로 착륙/정지하는 코드도 추가합니다.
    if keyboard.is_pressed('space'):
        print('착륙')
        drone.sendControlWhile(0, 0, 0, 0, 500) 
        drone.sendLanding()
        sleep(3)
        print('착륙 완료')
        is_takeoff = False
    if keyboard.is_pressed('q'):
        print('정지')
        drone.sendControlWhile(0, 0, 0, 0, 500)
        drone.sendStop()
        sleep(3)
        is_takeoff = False
    if cv2.waitKey(1) == 27: 
        break

capture.release()
cv2.destroyAllWindows()